In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [15]:
with open('worldwide_gross.txt', 'r', encoding='utf-8') as file:
    text = file.read()


In [16]:
soup = BeautifulSoup(text, 'html.parser')
table = soup.find('table')


In [17]:
full_data = []

for i, data in enumerate(table.find_all('tr')[1:], 1):
    temp = data.text.split('$')
    temp[0] = temp[0].replace(str(i), '')
    temp = [i] + temp

    temp[3] = temp[3].replace('<', '')

    if len(temp) == 4:
        temp[2] = temp[2].replace('-', '')
        temp.insert(3, '0,00000%')

    index = len(temp[3]) - temp[3][::-1].find(',') + 2
    temp.insert(3, temp[3][:index + 1])
    temp.insert(4, temp[4][index + 1:])
    temp.pop(5)
    temp.append(temp[-1][-4:])
    temp[5] = temp[5][:-4]

    index2 = len(temp[5]) - temp[5][::-1].find(',') + 2
    temp.insert(5, temp[5][:index2 + 1])
    temp.insert(6, temp[6][index2 + 1:])
    temp.pop(7)

    full_data.append(temp)


In [18]:
df = pd.DataFrame(full_data)
df.columns = ['Rank', 'Title', 'Worldwide_gross', 'Domestic_gross', 'Domestic %', 'Foreign_gross', 'Foreign %', 'Year']
df


,Rank,Title,Worldwide_gross,Domestic_gross,Domestic %,Foreign_gross,Foreign %,Year
0,1,Avatar,"2,923,710,708","785,221,649",26.9%,"2,138,489,059",73.1%,2009
1,2,Avengers: Endgame,"2,799,439,100","858,373,000",30.7%,"1,941,066,100",69.3%,2019
2,3,Avatar: The Way of Water,"2,320,250,281","684,075,767",29.5%,"1,636,174,514",70.5%,2022
3,4,Titanic,"2,264,812,968","674,354,882",29.8%,"1,590,458,086",70.2%,1997
4,5,Star Wars: Episode VII - The Force Awakens,"2,071,310,218","936,662,225",45.2%,"1,134,647,993",54.8%,2015
...,...,...,...,...,...,...,...,...
195,196,Madagascar: Escape 2 Africa,"603,900,354","180,010,950",29.8%,"423,889,404",70.2%,2008
196,197,War of the Worlds,"603,873,504","234,280,354",38.8%,"369,593,150",61.2%,2005
197,198,Tangled,"591,806,017","200,821,936",33.9%,"390,984,081",66.1%,2010
198,199,Quantum of Solace,"589,593,688","168,368,427",28.6%,"421,225,261",71.4%,2008


In [19]:
df.Worldwide_gross = df.Worldwide_gross.str.replace(',', '')
df.Worldwide_gross = df.Worldwide_gross.astype(np.int64)

df.Domestic_gross = df.Domestic_gross.str.replace(',', '')
df.Domestic_gross = df.Domestic_gross.astype(np.int64)

df['Domestic %'] = df['Domestic %'].str.replace('%', '').astype(float)

df.Foreign_gross = df.Foreign_gross.str.replace(',', '')
df.Foreign_gross = df.Foreign_gross.astype(np.int64)

df['Foreign %'] = df['Foreign %'].str.replace('%', '').astype(float)

df.Year = df.Year.astype(int)


In [20]:
df.to_excel('data.xlsx', index=False)
